[View in Colaboratory](https://colab.research.google.com/github/Dark-Sied/Word_embeddings_experiments/blob/master/harry_potter_embeddings.ipynb)

In [0]:
!pip install gensim

    100% |████████████████████████████████| 23.5MB 592kB/s 
    100% |████████████████████████████████| 1.4MB 12.3MB/s 
    100% |████████████████████████████████| 133kB 23.4MB/s 
    100% |████████████████████████████████| 61kB 17.0MB/s 
    100% |████████████████████████████████| 4.6MB 5.4MB/s 
    100% |████████████████████████████████| 552kB 19.7MB/s 
  Running setup.py bdist_wheel for smart-open ... - \ done
  Stored in directory: /root/.cache/pip/wheels/73/f1/9b/ccf93d4ba073b6f79b1ed9df68ab5ce048d8136d0efcf90b30
  Running setup.py bdist_wheel for bz2file ... - done
  Stored in directory: /root/.cache/pip/wheels/81/75/d6/e1317bf09bf1af5a30befc2a007869fa6e1f516b8f7c591cb9
Successfully built smart-open bz2file


In [0]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import re
from nltk.corpus import stopwords
from gensim.models import Word2Vec
import pandas as pd

In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
!git clone https://github.com/AmmarRashed/word_embeddings_hp.git

Cloning into 'word_embeddings_hp'...
remote: Counting objects: 76, done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 76 (delta 32), reused 37 (delta 6), pack-reused 0
Unpacking objects: 100% (76/76), done.


In [4]:
!ls

hinglish_new.w2v   hinglish_text.csv  word_embeddings_hp
HinglishSentiment  sample_data


In [5]:
%cd ./word_embeddings_hp/data/
!ls

/content/word_embeddings_hp/data
hp1.txt  hp2.txt  hp3.txt  hp4.txt  hp5.txt  hp6.txt  hp7.txt


In [6]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
stop_words = set(stopwords.words('english'))

In [0]:
ls = ["hp1.txt","hp2.txt","hp3.txt","hp4.txt","hp5.txt","hp6.txt", "hp7.txt"]

sent = []
for i in ls:
  with open(i) as f:
    text = f.readlines()
    for t in text:
      clean = re.sub(r'[^ a-z A-Z]','',t)
  #     print(clean)      
      a = sent_tokenize(clean)
      for j in a:
        sent.append(j)

    sent = list(filter(None, sent))
  #   print(sent)    
    english_df = pd.DataFrame(sent, columns = ["text"])
    english_df.to_csv("english_text.csv")

    

    words = [[w for w in s.lower().split() if w not in stop_words]for s in sent]
#     print(words)



In [9]:
model = Word2Vec(words, min_count = 40, size = 150, window = 10, workers = 1, sample = 1e-3)
print(model)

Word2Vec(vocab=2245, size=150, alpha=0.025)


In [10]:
model.most_similar("dumbledore")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('snape', 0.8824020028114319),
 ('slughorn', 0.8529797792434692),
 ('dumbledores', 0.7677643895149231),
 ('karkaroff', 0.7572901248931885),
 ('severus', 0.7570433020591736),
 ('umbridge', 0.7552511692047119),
 ('headmaster', 0.751039445400238),
 ('master', 0.7499511241912842),
 ('indeed', 0.7489374279975891),
 ('riddle', 0.7453637719154358)]

In [0]:
model.save("harry.w2v")